In [7]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import precision_recall_fscore_support
from scipy.sparse import hstack

In [6]:
# importing the data
datafolder = '../../data/'
train = datafolder+'train_with_features.csv'
test = datafolder+'test_with_features.csv'
dev = datafolder+'dev_with_features.csv'
# test_unseen = datafolder+'test_unseen_with_features.csv'
# dev_unseen = datafolder+'dev_unseen_with_features.csv'
df_train = pd.read_csv(train, skip_blank_lines=False)
df_dev = pd.read_csv(dev, skip_blank_lines=False)
# df_dev_unseen = pd.read_csv(dev_unseen, skip_blank_lines=False)
df_test = pd.read_csv(test, skip_blank_lines=False)
# df_test_unseen = pd.read_csv(test_unseen, skip_blank_lines=False)
# df = pd.concat([df_train, df_dev, df_test, df_dev_unseen, df_test_unseen], ignore_index=True, axis=0)
# df = df.fillna('')

In [5]:
df_dev.columns

Index(['id', 'img', 'label', 'text', 'tokens', 'lemmas', 'upos', 'pos_fw_emo',
       'count', 'emotion_associations', 'sentiment_score', 'intent',
       'ResNet_svm_rbf_kernel', 'ResNet_svm_rbf_kernelC10',
       'ResNet_svm_linear_kernel', 'svm_BoW_baseline_linear_C10',
       'Advanced_svm_linear_100K_C10', 'ResNet_svm_linear_kernelC10',
       'hatebert_direct', 'hatebert_vectors', 'bert_base_cased_finetuned',
       'ResNet_nn', 'baseline_svm_char_kernelC10'],
      dtype='object')

Block (c) concatenates the three decision vectors.Finally, 
block (d) is the classiﬁcation modulecomprised of four fully connected layers. 
    The sizeof the ﬁrst layer is equal to Ktimes the number ofclasses N.
    The second layer equals (4×N) which is greater than the size of the ﬁrst layer to again avoidthe Bottleneck phenomenon.
    The size of the thirdlayer is 2 ×N, which is associated with a dropout.
    The fourth (last) layer is a softmax function of asize equal to the number of classes.

In [13]:
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 1)) # to build 1n-grams from the word ==> BoW
                
X_train = vectorizer.fit_transform(df_train.tokens)
X_dev = vectorizer.transform(df_dev.tokens) 
X_test = vectorizer.transform(df_test.tokens)

Y_train = df_train.label.values
Y_dev = df_dev.label.values
Y_test = df_test.label.values

In [14]:
clf = LinearSVC(max_iter=1000000, C = 10,random_state =456)  # TODO: Hyperparms tuning
clf_svc = CalibratedClassifierCV(clf) 
# clf_svc.fit(X_train, Y_train)
# Y_pred = clf_svc.predict_pro(X_dev)

In [15]:
clf_svc.fit(X_train, Y_train)
# Y_pred = clf_svc.(X_dev)

CalibratedClassifierCV(base_estimator=LinearSVC(C=10, max_iter=1000000,
                                                random_state=456))

In [19]:
y_test_proba = clf_svc.predict_proba(X_test)

In [22]:
y_dev_proba = clf_svc.predict_proba(X_dev)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, otherwise use CPU

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(64 * 5 * 5, 4)
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv4(x))
        x = x.view(-1, 64 * 5 * 5)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create model and optimizer
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)